In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator
from copy import deepcopy
from tqdm import tqdm
import math

In [2]:
%matplotlib inline

In [3]:
train_data = pd.read_csv("./data/sonar_train.data", header=None).values
valid_data = pd.read_csv("./data/sonar_valid.data", header=None).values
test_data = pd.read_csv('./data/sonar_test.data', header=None).values

In [4]:
train_x, train_y = train_data[:,:-1], train_data[:,-1]
train_y = np.where(train_y == 2, -1.0, 1.0)
print(train_x.shape, train_y.shape)

valid_x, valid_y = valid_data[:,:-1], valid_data[:,-1]
valid_y = np.where(valid_y == 2, -1.0, 1.0)
print(valid_x.shape, valid_y.shape)

test_x, test_y = test_data[:,:-1], test_data[:,-1]
test_y = np.where(test_y == 2, -1.0, 1.0)
print(test_x.shape, test_y.shape)

(104, 60) (104,)
(52, 60) (52,)
(52, 60) (52,)


In [5]:
scaler = StandardScaler()
scaler = scaler.fit(train_x)

In [6]:
train_x = scaler.transform(train_x)
valid_x = scaler.transform(valid_x)
test_x = scaler.transform(test_x)

In [7]:
class L1(object):
    def __init__(self, coeff):
        self.coeff = coeff
        
    def __call__(self, w):
        return self.coeff*np.sum(np.abs(w))
    
    def grad(self, w):
        return self.coeff*np.sign(w)
    
class L2(object):
    def __init__(self, coeff):
        self.coeff = coeff
        
    def __call__(self, w):
        return 0.5*self.coeff*np.dot(w.T,w)[0]
    
    def grad(self, w):
        return self.coeff*w
    
class Sigmoid(object):
    def __call__(self, x):
        # Numerically stable sigmoid
        return np.where(x >= 0,
                    1 / (1 + np.exp(-x)),
                    np.exp(x) / (1 + np.exp(x)))
    
    def grad(self, x):
        return self(x)*(1-self(x))

In [8]:
class BestValue(object):
    def __init__(self, init_value=-np.inf, monitor_type="min"):
        assert monitor_type in ("min", "max")
        self.best_value = init_value
        self.monitor_type = monitor_type
        
    def update(self, value, model=None, **kwargs):
        if self.monitor_type == "max":
            if value > self.best_value:
                self.best_value = value
                self.additional_params = kwargs
                self.best_model = deepcopy(model)
        else:
            if value < self.best_value:
                self.best_value = value
                self.additional_params = kwargs
                self.best_model = deepcopy(model)

In [9]:
class LogisticRegression(BaseEstimator):
    # probabilistic approach
    # y must be 1 or -1
    def __init__(self, regularizer=None, coeff=0.0, lr=None, epochs=1000, valid_data=None, verbose=True):
        if regularizer == "l1" or isinstance(regularizer, L1):
            self.regularizer = L1(coeff)
        elif regularizer == "l2" or isinstance(regularizer, L2):
            self.regularizer = L2(coeff)
        elif regularizer is None:
            self.regularizer = None  # No regularization
        else:
            raise ValueError("regularizer must be one of 'l1', 'l2' or None. But got {}".format(regularizer))
        self.sigmoid = Sigmoid()
        self.lr = lr
        self.epochs = epochs
        self.best_train_accuracy = BestValue(init_value=0.0, monitor_type="max")
        self.valid_data = valid_data
        self.best_valid_accuracy = BestValue(init_value=0.0, monitor_type="max")
        self.verbose = verbose
        
    def __call__(self, x):
        if len(x.shape) == 1:
            x = np.expand_dims(x, axis=0)
        assert self.w.shape == (self.in_features, 1)
        y = np.dot(x, self.w) + self.b
        return np.squeeze(y, axis=1)
        
    def loss(self, x, y_true):
        losses = ((y_true + 1)*self(x)/2 - np.log(1+np.exp(self(x))))
        if self.regularizer is not None:
            return np.sum(losses) - self.regularizer(self.w)
        else:
            return np.sum(losses)
    
    def calculate_grad(self, x, y_true):
        self.db = np.sum((y_true+1)/2 - self.sigmoid(self(x)))
        if self.regularizer is not None:
            self.dw = np.sum(((y_true+1)/2 - self.sigmoid(self(x))).reshape(-1,1) * x, axis=0).reshape(-1, 1) - self.regularizer.grad(self.w)
        else:
            self.dw = np.sum(((y_true+1)/2 - self.sigmoid(self(x))).reshape(-1,1) * x, axis=0).reshape(-1, 1)
        
    def update(self, lr):
        # gradient ascent
        self.w = self.w + lr*self.dw
        self.b = self.b + lr*self.db
        
    def predict(self, x):
        y_pred = self.sigmoid(self(x))
        return np.where(y_pred>=0.5, 1, -1)
    
    def predict_best(self, x):
        best_model = self.best_valid_accuracy.best_model
        y_pred = self.sigmoid(best_model(x))
        return np.where(y_pred>=0.5, 1, -1)
        
    def calculate_accuracy(self, x, y):
        y_pred = self.predict(x)
        return np.mean(np.equal(y.astype(np.float32), y_pred.astype(np.float32)))
    
    def calculate_best_accuracy(self, x, y):
        y_pred = self.predict_best(x)
        return np.mean(np.equal(y.astype(np.float32), y_pred.astype(np.float32)))
    
    def score(self, x=None, y=None):
        # A bogus function for GridSearch to work
        if self.valid_data is None:
            return self.calculate_accuracy(x,y)
        else:
            return self.best_valid_accuracy.best_value
    
    def fit(self, train_x, train_y):
        self.in_features = train_x.shape[1]
        self.w = np.zeros((self.in_features, 1), dtype=np.float64)
        self.b = np.zeros(1, dtype=np.float64)[0]
        for i in range(1,self.epochs+1):
            train_loss = self.loss(x=train_x, y_true=train_y)
            self.calculate_grad(x=train_x, y_true=train_y)
            if self.lr is None:
                self.update(lr=2/(2+i))
            else:
                self.update(lr=self.lr)
            train_accuracy = self.calculate_accuracy(train_x, train_y)
            self.best_train_accuracy.update(train_accuracy, epoch=i, model = self, train_loss=train_loss)
            if self.valid_data:
                valid_loss = self.loss(x=self.valid_data[0],y_true=self.valid_data[1])
                valid_accuracy = self.calculate_accuracy(self.valid_data[0], self.valid_data[1])
                self.best_valid_accuracy.update(valid_accuracy, epoch=i, model = self, valid_loss=valid_loss)
            else:
                valid_accuracy=0.0
                valid_loss = -np.inf
            if self.verbose:
                print("epoch {}/{}: Train Loss: {}, Validation_loss: {} Training accuracy: {}, Validation_accuracy: {}".format(i, self.epochs, train_loss, valid_loss,train_accuracy, valid_accuracy))

# L2 regularization Grid search for lambda

In [10]:
best_accuracy = BestValue(init_value=0.0, monitor_type="max")
for lmbda in tqdm([math.pow(10,i) for i in range(-10,10)]):
    model = LogisticRegression(valid_data=(valid_x, valid_y), epochs=1000, verbose=False, regularizer="l2", coeff=lmbda)
    model.fit(train_x, train_y)
    best_accuracy.update(model.score(), model=model, coeff=lmbda)

 55%|█████████████████████████████████████████████                                     | 11/20 [00:04<00:03,  2.53it/s]C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in exp
C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in exp
C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp
 65%|█████████████████████████████████████████████████████▎                            | 13/20 [00:05<00:02,  2.56it/s]C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in multiply
  app.launch_new_instance()
C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher

In [11]:
best_accuracy.additional_params

{'coeff': 100.0}

In [12]:
best_accuracy.best_value

0.8846153846153846

In [13]:
best_model_l2_regularized = best_accuracy.best_model.best_valid_accuracy.best_model

In [14]:
best_model_l2_regularized.w.flatten()

array([-0.07203429, -0.03166687, -0.0050325 , -0.03503327, -0.0011076 ,
        0.01799703,  0.02137034,  0.00237779, -0.08505155, -0.07553474,
       -0.10091746, -0.10870769, -0.08369476, -0.01337836,  0.01927269,
        0.04159433,  0.04533289,  0.01636538, -0.03595391, -0.05750692,
       -0.0426478 , -0.03495508, -0.04103151, -0.02012037,  0.01024837,
        0.01660146, -0.00986387, -0.03063059, -0.02727321, -0.02284122,
        0.01138676, -0.01993036,  0.00643682,  0.04378844,  0.07081256,
        0.08809589,  0.07419586,  0.02212247,  0.0004961 ,  0.04591773,
        0.0437021 ,  0.00974678, -0.05601843, -0.06866506, -0.05369865,
       -0.03194237, -0.04522855, -0.06300298, -0.05832072,  0.00583477,
       -0.06516521, -0.07073703, -0.02344132, -0.03595537,  0.02591752,
       -0.03737281,  0.04061669, -0.04004346, -0.01883741, -0.01787745])

In [15]:
best_model_l2_regularized.b

-0.18515319257436305

In [16]:
test_accuracy = best_model_l2_regularized.calculate_accuracy(test_x, test_y)
print("Test accuracy for l2 regularized = {}".format(test_accuracy))

Test accuracy for l2 regularized = 0.7884615384615384


# L1 regularization grid search for lambda

In [17]:
best_accuracy_l1 = BestValue(init_value=0.0, monitor_type="max")
for lmbda in tqdm([math.pow(10,i) for i in range(-10,10)]):
    model = LogisticRegression(valid_data=(valid_x, valid_y), epochs=1000, verbose=False, regularizer="l1", coeff=lmbda)
    model.fit(train_x, train_y)
    best_accuracy_l1.update(model.score(), model=model, coeff=lmbda)

 60%|█████████████████████████████████████████████████▏                                | 12/20 [00:04<00:02,  3.06it/s]C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in exp
C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in exp
C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Acer\miniconda3\envs\ml_course\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  2.90it/s]


In [18]:
best_accuracy_l1.additional_params

{'coeff': 10.0}

In [19]:
best_accuracy_l1.best_value

0.9038461538461539

In [20]:
best_l1_model = best_accuracy_l1.best_model.best_valid_accuracy.best_model

In [21]:
best_l1_model.calculate_accuracy(test_x, test_y)

0.7307692307692307

In [22]:
best_l1_model.w.flatten()

array([-0.01451205, -0.00562126, -0.01720734, -0.02977406, -0.03703174,
        0.00634904,  0.02738394, -0.00334069, -0.02465461, -0.03212977,
       -0.23064254, -0.17837461, -0.0015372 , -0.00514361,  0.01266242,
        0.02507199,  0.0070222 , -0.0101527 , -0.02832369, -0.07241257,
       -0.01449742, -0.03784816, -0.06239183, -0.02209457,  0.01491925,
        0.01778565, -0.02055214, -0.00291438, -0.0076785 , -0.02606921,
        0.02545161, -0.01658357,  0.01424691,  0.0110537 ,  0.0261303 ,
        0.10989975,  0.04939839,  0.00630216,  0.02633256,  0.06555013,
        0.00885936,  0.02552834, -0.0469365 , -0.02401955,  0.00472317,
       -0.01095853, -0.044027  , -0.01265277, -0.01458956,  0.02128773,
       -0.02102451, -0.01474456, -0.0298286 ,  0.01532888,  0.04309554,
       -0.01982156,  0.01864155, -0.02562613,  0.02456804, -0.03329605])

In [23]:
best_l1_model.b

-0.0531693021143948

# Without regularization

In [24]:
model = LogisticRegression(valid_data=(valid_x, valid_y), epochs=1000, verbose=False, regularizer=None)
model.fit(train_x, train_y)

In [25]:
model.best_valid_accuracy.best_value

0.8461538461538461

In [26]:
model.calculate_best_accuracy(test_x, test_y)

0.7884615384615384